In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [736 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,079 kB]
Get:13 http://security.ubu

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").config("spark.driver.memory", "2g").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
sales_df.dtypes


[('id', 'string'),
 ('date', 'string'),
 ('date_built', 'string'),
 ('price', 'string'),
 ('bedrooms', 'string'),
 ('bathrooms', 'string'),
 ('sqft_living', 'string'),
 ('sqft_lot', 'string'),
 ('floors', 'string'),
 ('waterfront', 'string'),
 ('view', 'string')]

In [ ]:
# 2. Create a temporary view of the DataFrame.
sales_df.createOrReplaceTempView('home_sales')

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""SELECT ROUND(AVG(price),2), YEAR(date)
             from home_sales
             where bedrooms == 4
             group by YEAR(date)
             ORDER BY YEAR(date) desc""").show(truncate=False)

+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|296363.88           |2022      |
|301819.44           |2021      |
|298353.78           |2020      |
|300263.7            |2019      |
+--------------------+----------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""SELECT ROUND(AVG(price),2), YEAR(date)
             from home_sales
             where bedrooms == 4 AND bathrooms == 3
             group by YEAR(date)
             ORDER BY YEAR(date) desc""").show(truncate=False)


+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|290780.14           |2022      |
|292627.68           |2021      |
|289659.97           |2020      |
|292438.24           |2019      |
+--------------------+----------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""SELECT ROUND(AVG(price),2), YEAR(date)
             from home_sales
             where bedrooms == 4 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000
             group by YEAR(date)
             ORDER BY YEAR(date) desc""").show(truncate=False)


+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|295505.36           |2022      |
|296672.69           |2021      |
|289891.27           |2020      |
|301672.15           |2019      |
+--------------------+----------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()

spark.sql("""SELECT ROUND(AVG(price),2), view
             from home_sales
             where price >= 350000
             group by view
             ORDER BY CAST(view AS int) desc""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|1026669.5           |100 |
|1061201.42          |99  |
|1053739.33          |98  |
|1129040.15          |97  |
|1017815.92          |96  |
|1054325.6           |95  |
|1033536.2           |94  |
|1026006.06          |93  |
|970402.55           |92  |
|1137372.73          |91  |
|1062654.16          |90  |
|1107839.15          |89  |
|1031719.35          |88  |
|1072285.2           |87  |
|1070444.25          |86  |
|1056336.74          |85  |
|1117233.13          |84  |
|1033965.93          |83  |
|1063498.0           |82  |
|1053472.79          |81  |
+--------------------+----+
only showing top 20 rows

--- 0.9422791004180908 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
start_time = time.time()

spark.sql("""SELECT ROUND(AVG(price),2), view
             from home_sales
             where price >= 350000
             group by view
             ORDER BY CAST(view AS int) desc""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|1026669.5           |100 |
|1061201.42          |99  |
|1053739.33          |98  |
|1129040.15          |97  |
|1017815.92          |96  |
|1054325.6           |95  |
|1033536.2           |94  |
|1026006.06          |93  |
|970402.55           |92  |
|1137372.73          |91  |
|1062654.16          |90  |
|1107839.15          |89  |
|1031719.35          |88  |
|1072285.2           |87  |
|1070444.25          |86  |
|1056336.74          |85  |
|1117233.13          |84  |
|1033965.93          |83  |
|1063498.0           |82  |
|1053472.79          |81  |
+--------------------+----+
only showing top 20 rows

--- 0.5386006832122803 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [ ]:
# 11. Read the parquet formatted data.
p_sales_df=spark.read.parquet('home_sales_partitioned')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_sales_df.createOrReplaceTempView('p_home_sales')

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
start_time = time.time()

spark.sql("""SELECT ROUND(AVG(price),2), view
             from p_home_sales
             where price >= 350000
             group by view
             ORDER BY CAST(view AS int) desc""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|1026669.5           |100 |
|1061201.42          |99  |
|1053739.33          |98  |
|1129040.15          |97  |
|1017815.92          |96  |
|1054325.6           |95  |
|1033536.2           |94  |
|1026006.06          |93  |
|970402.55           |92  |
|1137372.73          |91  |
|1062654.16          |90  |
|1107839.15          |89  |
|1031719.35          |88  |
|1072285.2           |87  |
|1070444.25          |86  |
|1056336.74          |85  |
|1117233.13          |84  |
|1033965.93          |83  |
|1063498.0           |82  |
|1053472.79          |81  |
+--------------------+----+
only showing top 20 rows

--- 1.563511848449707 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False